In [21]:
import numpy as np
import cv2
import os
import glob
import re
import pandas as pd
import yaml
import csv
from scipy.spatial.transform import Rotation as R

# ================= 1. 配置区域 =================

# 路径配置
IMAGE_FOLDER = "/home/lgx/Project/hand_eye_calib/src/calibrate/raw_data"
OUTPUT_CSV = os.path.join(IMAGE_FOLDER, 'vision_poses.csv')
CAMERA_MATRIX_PATH = '/home/lgx/.ros/camera_info/rgb_camera.yaml'

# 相机内参加载
with open(CAMERA_MATRIX_PATH, 'r') as f:
    cam_info = yaml.safe_load(f)
CAMERA_MATRIX = np.array(cam_info['camera_matrix']['data']).reshape(3,3)
DIST_COEFFS = np.array(cam_info['distortion_coefficients']['data'])

# 棋盘格参数 (请务必确认内部角点数)
CHESSBOARD_SIZE = (11, 8)  # (columns, rows)
CHESSBOARD_SQUARE_SIZE = 0.01  # 每个格子的边长 (米)

print('CAMERA_MATRIX:\n', CAMERA_MATRIX)
print('DIST_COEFFS:\n', DIST_COEFFS)

CAMERA_MATRIX:
 [[525.56704028   0.         324.87738252]
 [  0.         526.40438798 256.39401889]
 [  0.           0.           1.        ]]
DIST_COEFFS:
 [ 0.01374356 -0.13993062 -0.00498286  0.00522853  0.        ]


In [23]:
# ================= 2. 辅助函数 =================

def extract_index(filename):
    """从文件名提取数字序号，例如 frame_0005.png -> 5"""
    nums = re.findall(r'\d+', filename)
    return int(nums[0]) if nums else 0

# ================= 3. 主处理循环 =================

def process_and_select_poses():
    # 预计算 3D 物体点 (棋盘格坐标系原点在第一个角点)
    # objp 的顺序遵循先遍历列再遍历行的原则，以匹配 solvePnP
    objp = np.zeros((CHESSBOARD_SIZE[0] * CHESSBOARD_SIZE[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2) * CHESSBOARD_SQUARE_SIZE

    # 获取并按序号排序图片
    image_files = sorted(glob.glob(os.path.join(IMAGE_FOLDER, "*.png")), key=extract_index)
    
    results = []
    print(f"检测到 {len(image_files)} 张图片。正在进入交互模式...")
    print("操作指南: [y] 接受并保存 | [n] 放弃此图 | [ESC] 退出程序")

    for img_path in image_files:
        filename = os.path.basename(img_path)
        img_index = extract_index(filename)
        
        frame = cv2.imread(img_path)
        if frame is None: continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 1. 查找角点
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD_SIZE, 
            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)
        
        if ret:
            # 2. 亚像素精修，确保位姿解算达到 0.1 像素精度
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
            corners_sub = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)
            
            # 3. 可视化四个角落的点
            cols, rows = CHESSBOARD_SIZE
            indices = [0, cols-1, (rows-1)*cols, rows*cols-1]
            labels = ["TL[0]", f"TR[{cols-1}]", f"BL[{(rows-1)*cols}]", f"BR[{rows*cols-1}]"]
            
            vis_frame = frame.copy()
            for i, idx in enumerate(indices):
                pt = corners_sub[idx].ravel()
                x, y = int(pt[0]), int(pt[1])
                # cv2.circle(vis_frame, (x, y), , (0, 255, 0), 2)  # 绿色外圈
                cv2.circle(vis_frame, (x, y), 1, (0, 0, 255), -1) # 红色中心
                cv2.putText(vis_frame, labels[i], (x + 10, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 3, cv2.LINE_AA)
                cv2.putText(vis_frame, labels[i], (x + 10, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 1, cv2.LINE_AA)

            # 显示窗口
            cv2.imshow('Selection: Y/N?', cv2.resize(vis_frame, (1280, 960)))
            key = cv2.waitKey(0)

            if key == ord('y') or key == ord('Y'):
                # 4. 高精度求解 PnP
                # 注意：直接在原始图像上使用原始畸变系数解算以保留最高精度
                success, rvec, tvec = cv2.solvePnP(objp, corners_sub, CAMERA_MATRIX, DIST_COEFFS, flags=cv2.SOLVEPNP_ITERATIVE)
                
                if success:
                    # 使用 LM 算法精修
                    rvec, tvec = cv2.solvePnPRefineLM(objp, corners_sub, CAMERA_MATRIX, DIST_COEFFS, rvec, tvec)
                    
                    tx, ty, tz = tvec.flatten()
                    rx, ry, rz = rvec.flatten()
                    results.append([img_index, tx, ty, tz, rx, ry, rz])
                    print(f"✅ 已保存 Index {img_index}")
            elif key == 27: # ESC 键
                break
            else:
                print(f"❌ 已跳过 Index {img_index}")
        else:
            print(f"⚠ 未能识别棋盘格: {filename}")

    cv2.destroyAllWindows()

    # 5. 保存至 CSV
    if results:
        df = pd.DataFrame(results, columns=['img_index', 'cam_tx', 'cam_ty', 'cam_tz', 'cam_rx', 'cam_ry', 'cam_rz'])
        df = df.sort_values(by='img_index')
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"\n🎉 处理完成！成功提取 {len(results)} 组位姿数据。")
        print(f"CSV 路径: {OUTPUT_CSV}")
    else:
        print("\n❌ 未能提取任何数据。")

# 执行程序
if __name__ == "__main__":
    process_and_select_poses()

检测到 25 张图片。正在进入交互模式...
操作指南: [y] 接受并保存 | [n] 放弃此图 | [ESC] 退出程序
✅ 已保存 Index 0
✅ 已保存 Index 1
✅ 已保存 Index 2
✅ 已保存 Index 3
✅ 已保存 Index 4
✅ 已保存 Index 5
✅ 已保存 Index 6
✅ 已保存 Index 7
✅ 已保存 Index 8
✅ 已保存 Index 9
✅ 已保存 Index 10
✅ 已保存 Index 11
✅ 已保存 Index 12
✅ 已保存 Index 13
✅ 已保存 Index 14
✅ 已保存 Index 15
✅ 已保存 Index 16
✅ 已保存 Index 17
⚠ 未能识别棋盘格: frame_0018.png
✅ 已保存 Index 19
✅ 已保存 Index 20
✅ 已保存 Index 21
✅ 已保存 Index 22
✅ 已保存 Index 23
✅ 已保存 Index 24

🎉 处理完成！成功提取 24 组位姿数据。
CSV 路径: /home/lgx/Project/hand_eye_calib/src/calibrate/raw_data/vision_poses.csv
